# 신경망 학습

In [1]:
import numpy as np
from dataset.mnist import load_mnist

## Helper functions

In [3]:
# 손실함수 (Cross Entropy)
def cross_entropy_error(y, t):
    delta = 1e-7 # 0.0000001
    
    if y.ndim == 1:
        t = t.reshape(1, t.size) 
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y+delta))/batch_size

In [4]:
# 신경망에서 사용할 W(Matrix 형태)의 편미분 행렬을 구하는 함수
# 신경망의 기울기 : 그레디언트 (편미분 벡터)
def numerical_gradient(f, x): # x의 shape이 (784, 20) => grads 도 (784, 20)
    h = 1e-4 # 0.0001
    grads = np.zeros_like(x)
    
    it = np.nditer(x, flags=["multi_index"], op_flags=["readwrite"])
        
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]

        x[idx] = tmp_val + h
        fxh1 = f(x) # f(x+h)
        x[idx] = tmp_val - h
        fxh2 = f(x) # f(x-h)
    
        grads[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
        it.iternext()
        
    return grads

In [5]:
# Softmax
def softmax(x):
    if x.ndim == 2:
        x = x.T # 10*100
        x = x - np.max(x, axis=0) # 10*100 - 100 = 10*100
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

In [6]:
# Sigmoid
def sigmoid(x):
    return 1/(1+np.exp(-x))

## 2층 신경망 구현하기

In [7]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # W1 shape (784, 20), B1 shape (20, ), W2 shape (20, 10). B2 shape (10,)
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        #todo
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1 # (784,) X (784, 20) + (20,)
        z1 = sigmoid(a1) # (20,)
        
        a2 = np.dot(z1, W2) + b2 # (20,) X (20, 10) + (10,)
        y = softmax(a2) # (10,)
        
        return y
    
    def loss(self, x, t):
        #todo
        y = self.predict(x)
        loss = cross_entropy_error(y, t)
        return loss      
    
    def accuracy(self, x, t):
        #todo
        y = self.predict(x) # y는 0~1사이의 확률값
        y = np.argmax(y, axis=1) # y는 클래스의 인덱스
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y==t)/x.shape[0]
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        f = lambda w : self.loss(x, t)
        
        #todo
        # W1 shape (784, 20), B1 shape (20, ), W2 shape (20, 10). B2 shape (10,)
        grads = {}
        grads["W1"] = numerical_gradient(f, self.params['W1']) # W1 (784, 20) --> dW1 (784, 20)
        grads["b1"] = numerical_gradient(f, self.params['b1']) # b1 (20,) --> db1 (20,)
        grads["W2"] = numerical_gradient(f, self.params['W2']) # W2 (20, 10) --> dW2 (20, 10)
        grads["b2"] = numerical_gradient(f, self.params['b2']) # b2 (10,) --> db2 (10,)
        
        return grads

In [8]:
(X_train, y_train ), (X_test, y_test) = load_mnist(normalize=True, one_hot_label=True)

In [ ]:
network = TwoLayerNet(input_size=784, hidden_size=20, output_size=10)

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 하이퍼 파라미터
iters_num = 10000
train_size = X_train.shape[0] # 60000개
batch_size = 100 # 미니 배치 사이즈
learning_rate = 0.1

# 1 에폭(epoch)당 반복수 
# 60000개의 훈련 데이터를 100개씩 학습했을때 다 소진했을때 반복횟수 = 600번
iter_per_epoch = train_size/batch_size # 600번 = 1 epoch

for i in range(iters_num): 
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = X_train[batch_mask]
    t_batch = y_train[batch_mask]
    
    # 1. Gradient
    grads = network.numerical_gradient(x_batch, t_batch)
        
    # 2. Gradient Descent
    for keys in ("W1", "W2", "b1", "b2"):
        # W(new) <-- W(old) - (lr * Gradient) : 경사하강법
        network.params[keys] = network.params[keys] - (learning_rate * grads[keys])
    
    loss = network.loss(x_batch, t_batch) # 산
    train_loss_list.append(loss)
    print(i, loss)

    if i % iter_per_epoch == 0:  # 600, 1200, 1800 (1 epoch 마다...)
        train_accuracy = network.accuracy(X_train, y_train)
        test_accuracy = network.accuracy(X_test,y_test)
        train_acc_list.append(train_accuracy)
        test_acc_list.append(test_accuracy)
        print("Train Accuracy :" + str(train_accuracy) + "Test Accuracy:" + str(test_accuracy))

0 2.296560032679572
Train Accuracy :0.09736666666666667Test Accuracy:0.0982
1 2.2981195224739603
2 2.302989881568066
3 2.297146595719491
4 2.296945242622537
5 2.2949501133259758
6 2.297733430119844
7 2.298243622833307
8 2.2886090748895436
9 2.285692289416506
10 2.3017051121153718
11 2.279963793752531
12 2.3062615077230206
13 2.303800828443942
14 2.2999217702042496
15 2.2981558399932
16 2.301968135169615
17 2.3049267173221035
18 2.2966315717930326
19 2.2986318482988186
20 2.28836816665131
21 2.2901644019289242
22 2.3032438910508968
23 2.3040939714241295
24 2.296718891149811
25 2.2868567373581428
26 2.2929482502882625
27 2.295983536328075
28 2.2841488884280023
29 2.2990958319521075
30 2.30539192745004
31 2.3014852472946394
32 2.302467434818294
33 2.300734336097548
34 2.298236821522961
35 2.300902658538451
36 2.290690641827391
37 2.295464512755928
38 2.2997299481982334
39 2.3038942546045926
40 2.289998882388407
41 2.2974615962435867
42 2.2915168902896625
43 2.287919522518506
44 2.29940898

366 1.6962831061499537
367 1.7575118190822479
368 1.6555871395097888
369 1.715834366046493
370 1.7700864471867142
371 1.7279100922159063
372 1.7057935953554004
373 1.6806795779562582
374 1.7492795547182316
375 1.7252146109820539
376 1.6388532707416374
377 1.6509706733046472
378 1.6635494838839935
379 1.731190144100781
380 1.6836087828951107
381 1.6778953884663974
382 1.672625753913724
383 1.6310171928651982
384 1.599760452884829
385 1.6811999131690862
386 1.6758386469622497
387 1.7019984088430045
388 1.6812410805738525
389 1.679181650145777
390 1.6364401731677283
391 1.7298666395281574
392 1.72021859898246
393 1.6148079690866535
394 1.600905220992612
395 1.6125967467039362
396 1.5240210970742616
397 1.5986743957854876
398 1.6448670756494763
399 1.5883730050591538
400 1.561573210106823
401 1.6102140919352348
402 1.6534763086848099
403 1.6221415214525388
404 1.6691975732015027
405 1.6489908318133235
406 1.5557080661044915
407 1.6233201484463269
408 1.5124320768355484
409 1.60650770436001

724 0.9618092008110216
725 0.9574725858794468
726 0.7602352568376313
727 0.8780164492591348
728 0.8906485567981656
729 0.9136002747636021
730 0.8524897707041451
731 0.7942918570371243
732 0.797471512197617
733 0.8770517362387885
734 0.8757822705595464
735 0.7989492024706988
736 0.94589763495804
737 0.8655057637762232
738 0.8963132717890894
739 0.818532189876651
740 0.8615559926494061
741 0.8080609946292825
742 0.8605527851950555
743 0.7707706690645092
744 0.9590197182652653
745 0.8893104929734619
746 0.9114740296596682
747 0.9073957821987206
748 0.8629873136546283
749 0.7885012953566425
750 0.8033684285699663
751 0.836372153374422
752 0.9225404692897172
753 0.7531708176507469
754 0.8085565336683603
755 0.8134635338107624
756 0.8524999769684691
757 0.8846452272368083
758 0.8630344245724291
759 0.8979695547282303
760 0.8214939145961984
761 0.8928814216681294
762 0.8960610056493263
763 0.8585746763299915
764 0.9168586243138159
765 0.788933769581475
766 0.8216472630018876
767 0.71844729591

1079 0.406411508229577
1080 0.6210764383685584
1081 0.6223124819569879
1082 0.5773471395354958
1083 0.6141251853102566
1084 0.635544430289238
1085 0.5914906165946741
1086 0.5822064524063301
1087 0.63163042964574
1088 0.5809448706736553
1089 0.6303679199594963
1090 0.4602648748562157
1091 0.602340550079813
1092 0.6714540904098217
1093 0.5506144122065061
1094 0.5855399403760728
1095 0.5237454730318213
1096 0.5954794966349154
1097 0.6039476264026844
1098 0.5074165940782247
1099 0.6369372513953363
1100 0.5594469436093372
1101 0.5835096192519797
1102 0.6884598928927965
1103 0.5806496519020229
1104 0.5967835812603672
1105 0.6163409136865137
1106 0.639279347994723
1107 0.6391973769420207
1108 0.5586550710000368
1109 0.5776651705986567
1110 0.5339803058674487
1111 0.5975151988904798
1112 0.5144636052723126
1113 0.6612440636508267
1114 0.669246178710118
1115 0.6172181532649523
1116 0.5701124183636254
1117 0.6597845523562532
1118 0.5596010689758909
1119 0.5433640123372997
1120 0.5936826143343862

1418 0.4503414906567798
1419 0.49660985591853296
1420 0.5556784714892913
1421 0.5100032083969691
1422 0.5003188594740614
1423 0.5829794593623128
1424 0.6386401705453626
1425 0.4724000694648517
1426 0.4836579009031341
1427 0.4852696699080127
1428 0.4907395860946362
1429 0.5007824050603716
1430 0.43841813931973006
1431 0.5671768327195763
1432 0.43363703436942
1433 0.5341386186172532
1434 0.38212863610860937
1435 0.4423191180464272
1436 0.35585047016983506
1437 0.39837292814652087
1438 0.5600251784638179
1439 0.45601801676351555
1440 0.4533083929756217
1441 0.43662332570086904
1442 0.5652770319354835
1443 0.6003679402305336
1444 0.5199397958952431
1445 0.535896939639413
1446 0.5133378695449576
1447 0.6386852927401839
1448 0.4386998697253473
1449 0.42249001273910486
1450 0.5071296340337106
1451 0.4849653104299947
1452 0.391927410324257
1453 0.48140877957948885
1454 0.560975798143498
1455 0.615277910790558
1456 0.4902603155220038
1457 0.578894289678546
1458 0.4832377860502906
1459 0.4629669

...

6435 0.20851429276740022
6436 0.28511681221176305
6437 0.30336742058942484
6438 0.18984006398038245
6439 0.27288855293232106
6440 0.37838371353743455
6441 0.19670303885297272
6442 0.13108136729289613
6443 0.2859710741069648
6444 0.25034196128660163
6445 0.2072336180671623
6446 0.28310903368374724
6447 0.1374346032539583
6448 0.2467030040894599
6449 0.19468391141179653
6450 0.2711547914400692
6451 0.20517563137284242
6452 0.22304621597217295
6453 0.11782549392265207
6454 0.2675652022665037
6455 0.21237382407063113
6456 0.21845703256970836
6457 0.2930034157844471
6458 0.2205450789188131
6459 0.17909095695135074
6460 0.1992089515280332
6461 0.2606115284709597
6462 0.2658735942562548
6463 0.27571102390870045
6464 0.22460850347863276
6465 0.2044727759722665
6466 0.21465749536970638
6467 0.21129945559761032
6468 0.2807204993862497
6469 0.19113929648846054
6470 0.2010219597626617
6471 0.15330907557450954
6472 0.19724382338532545
6473 0.1831392325499418
6474 0.18152921060390567
6475 0.16742410

6767 0.1838742712554946
6768 0.18329198748372405
6769 0.24633475301349697
6770 0.18630146600227485
6771 0.289208281163889
6772 0.17079378410761537
6773 0.2049196964833056
6774 0.19213986290336885
6775 0.32574634388126356
6776 0.22178154241217293
6777 0.25686054903822514
6778 0.15153059144200542
6779 0.23922201589652745
6780 0.3168727438437973
6781 0.2136307410004225
6782 0.25942181645119305
6783 0.1730891048053189
6784 0.23122214804554037
6785 0.23640912274519288
6786 0.18348819427939256
6787 0.2794142544396984
6788 0.16538602439655845
6789 0.23209425161239333
6790 0.14048692154705544
6791 0.34310798654893787
6792 0.20959441898907719
6793 0.31332868989959395
6794 0.3729992135039317
6795 0.31441469358918267
6796 0.2811157248111014
6797 0.31382096181325625
6798 0.24759239294714036
6799 0.2987597097803241
6800 0.16737811095934446
6801 0.2573152624710843
6802 0.16363293591890826
6803 0.2823214325545114
6804 0.2634206614456297
6805 0.1944512328585448
6806 0.20711675418902684
6807 0.23841150

7101 0.28737325418097576
7102 0.2078472257464035
7103 0.17473268265975012
7104 0.2045875707274404
7105 0.2597155872040274
7106 0.17969390493296308
7107 0.25394672123931983
7108 0.24943011239315357
7109 0.4114673731518006
7110 0.3319048957218112
7111 0.2730469335084091
7112 0.19390114020994098
7113 0.14156689457679777
7114 0.06919400701608458
7115 0.21706036157423228
7116 0.27279838049759614
7117 0.22689349952054993
7118 0.3082759576213273
7119 0.1385887770262098
7120 0.13912510057334532
7121 0.19081263367373033
7122 0.20632564531976683
7123 0.15022110538709224
7124 0.2415608422372067
7125 0.1707143858374637
7126 0.16645268789906978
7127 0.2684969146135131
7128 0.22064880012476906
7129 0.2944473946981018
7130 0.23048191981268779
7131 0.12402357987765868
7132 0.1861267048762283
7133 0.1805546006361275
7134 0.18812811594747217
7135 0.2012648908684006
7136 0.15549392526038117
7137 0.18845872256666144
7138 0.13177003264743498
7139 0.17198742898908523
7140 0.1887306376537481
7141 0.279519150

7433 0.267568331296981
7434 0.16178036982711588
7435 0.15107106688902025
7436 0.192259042236803
7437 0.29404931269947876
7438 0.19262726900205457
7439 0.16027817763684163
7440 0.20859275517847817
7441 0.128582121967702
7442 0.20120110258558263
7443 0.2011671988076726
7444 0.30947848169024916
7445 0.17946650247625018
7446 0.19800159543051685
7447 0.2934060896177648
7448 0.22272721568388298
7449 0.14248451310980287
7450 0.21116973041592849
7451 0.16800982788586583
7452 0.2797586852556128
7453 0.1797947075437214
7454 0.2070228538065647
7455 0.12655828022627066
7456 0.28289718403182385
7457 0.19633295708290874
7458 0.31272041021658964
7459 0.18379583960517296
7460 0.19574989236866658
7461 0.136086342981519
7462 0.18945129863295573
7463 0.11039843712845154
7464 0.203300410604022
7465 0.2810740520848115
7466 0.21058611361900326
7467 0.250386999722956
7468 0.16646981298410687
7469 0.26946522983910776
7470 0.16196901412713427
7471 0.11995441033029289
7472 0.12114240748079581
7473 0.33883469012

7767 0.1729321854182664
7768 0.20490540260973977
7769 0.14752666927343033
7770 0.26596428478875067
7771 0.2713884503181807
7772 0.17358349901783526
7773 0.19976800595063765
7774 0.21179059780419407
7775 0.176695268934453
7776 0.22296830435337145
7777 0.17213628726756988
7778 0.07613891465314909
7779 0.22371435169225024
7780 0.24013065178143767
7781 0.2639120355254234
7782 0.17274048177376247
7783 0.17836218761803443
7784 0.2699207221441213
7785 0.2854400573197172
7786 0.2873540572670577
7787 0.2941418596437412
7788 0.17436442671807842
7789 0.26909188756653285
7790 0.1568864878706108
7791 0.1986524240508025
7792 0.1885776859451881
7793 0.25790967379902413
7794 0.2114848035041897
7795 0.12484331562487236
7796 0.21536506697439883
7797 0.2935414566841251
7798 0.1670504215179358
7799 0.2517031267892225
7800 0.17210043886058382
Train Accuracy :0.9374Test Accuracy:0.9355
7801 0.30505742189666535
7802 0.12898595832572757
7803 0.237489847307872
7804 0.23559504769244924
7805 0.23677362365681592
